In [1]:
import numpy as np
import pandas as pd
import os, glob
import nibabel as nib

In [2]:
# Call 109 subjects for seg
seg_dir = 'segmentation images directory'
prob_dir = 'Directory for probaility maps for each institution'

segm_dir = sorted(glob.glob(os.path.join(seg_dir, "*_seg*.nii.gz"))) 
probailitymap_dir = sorted(glob.glob(os.path.join(prob_dir, "*_noPool*.nii.gz"))) 

In [3]:
# Load nifti files
def LoadingImage(dir):
    
    nifti_image = nib.load(dir)
    image = np.asarray(nifti_image.dataobj)
    header = nifti_image.header
    imgaffine = nifti_image.affine
    
    return image, header, imgaffine

In [4]:
# Separately collects unique subject name from files name
def getFilename(full_dir):
    _,filename = full_dir.split('\\')
    print(filename)
    subject,_,_,_,_= filename.split('_')
    return str(subject)

In [5]:
# Smoothing the probaility map by averaging around center voxel
# apply average only counting values in edema
def patch_average(image, seg_labels, label=2, patch_size=(5,5,5)):
    # Get the dimensions of the image
    dim_x, dim_y, dim_z = image.shape
    p_x, p_y, p_z = [size // 2 for size in patch_size]  # Size of patch in each dimension

    # Find voxels with the given label
    voxels = np.array(np.where(seg_labels == label)).T

    # Create a copy of the original image to avoid altering it
    avg_image = image.copy()

    # Iterate over these voxels, calculate the averages of the patches and replace the voxel values
    for voxel in voxels:
        x, y, z = voxel
        # Define the patch boundaries while taking care of the image boundaries
        x_start, x_end = max(0, x - p_x), min(dim_x, x + p_x + 1)
        y_start, y_end = max(0, y - p_y), min(dim_y, y + p_y + 1)
        z_start, z_end = max(0, z - p_z), min(dim_z, z + p_z + 1)

        # Extract the image patch
        img_patch = image[x_start:x_end, y_start:y_end, z_start:z_end]

        # Extract the corresponding segmentation patch
        seg_patch = seg_labels[x_start:x_end, y_start:y_end, z_start:z_end]

        # Mask the image patch with the segmentation patch
        masked_patch = img_patch[seg_patch == label]

        # Calculate the average of the masked patch
        patch_mean = masked_patch.mean()

        # If the average is zero, add 0.05
        if patch_mean == 0:
            patch_mean += 0.05

        # Replace the voxel value
        avg_image[x, y, z] = patch_mean

    return avg_image

In [ ]:
for i in range(len(segm_dir)):
    # Seg (240, 240, 155)
    seg_img, seg_hdr, seg_imgaffine = LoadingImage(segm_dir[i])
    # binary (240, 240, 155)
    binary_img, binary_hdr, binary_imgaffine = LoadingImage(probailitymap_dir[i])

    # Average smoothing
    avg_image = patch_average(binary_img, seg_img)

    # Binarize the data
    binary_image = np.where(avg_image > 0.8, 1, 0)

    # filename
    filename = getFilename(segm_dir[i])

    # Save the output
    nifti_img = nib.Nifti1Image(binary_image, seg_imgaffine, seg_hdr)
    nib.save(nifti_img, f'save it into desired output directory')